In [ ]:
from grid.eval import *

In [ ]:
worlds, agents = worlds_and_agents(3)

In [ ]:
n_agents = len(agents)
wins  = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
games = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
soln = None

In [ ]:
while True:
    soln = activelo.solve(games, wins, soln=soln)
    sugg = activelo.suggest(soln)

    results = arena.common.evaluate(worlds, {n: agents[n] for n in sugg})
    games, wins = update(games, wins, results)

    report(soln)

In [ ]:
plt.plot(soln.μ)

In [ ]:
params = {}
for idx, row in snapshots(3).iterrows():
    s = storage.load_snapshot(row.run, row.idx)
    params[idx] = {**runs.info(row.run)['params'], 'samples': s['n_samples'], 'flops': s['n_flops']}
params = pd.DataFrame.from_dict(params, orient='index')

In [ ]:
snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
snaps['params'] = params(snaps)

In [ ]:
import plotnine as pn
(pn.ggplot(data=snaps)
    + pn.geom_line(pn.aes(x='flops', y='μ', group='run', color='params'))
    + pn.scale_x_continuous(trans='log10'))